# Model 1

## Pakete und Daten laden

In einem ersten Schritt laden wir die nöten Pakete und laden die Daten.

In [9]:
import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer
import os
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance
from pathlib import Path

target = Path("C:/Users/mhu/Documents/github/topic_model_it")
os.chdir(target)

data_model_1 = pd.read_csv("data/informatikkurse.csv")
print(data_model_1.head())

   Unnamed: 0  veranstaltung_id  \
0           1           1717848   
1           2           1717914   
2           3           1717915   
3           4           1717989   
4           5           1717990   

                                 veranstaltung_titel kursbeschreibung  \
0    Übung zu: Didaktische Grundlagen für Informatik              NaN   
1  Seminar zu: Lehren und Lernen im Fach Informat...              NaN   
2                  Übung zu: Ethik in der Informatik              NaN   
3                  Übung zu: Ethik in der Informatik              NaN   
4                  infSP2F-01a: Softwareprojekt (2F)              NaN   

  kursformat_original sprache_original lernziele lernmethode literatur  \
0               Übung              NaN       NaN         NaN       NaN   
1             Seminar              NaN       NaN         NaN       NaN   
2               Übung              NaN       NaN         NaN       NaN   
3               Übung              NaN       NaN      

C:\Users\mhu\AppData\Local\Temp\ipykernel_1604\1891781394.py:25: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data_model_1 = pd.read_csv("data/informatikkurse.csv")


## Reproduzierbarkeit durch Seeds
Im folgenden fixieren wir die Zufallszahlen-Generatoren aller beteiligten Bibliotheken auf den Wert 11. Dies stellt sicher, dass die Experimente bei jedem Durchlauf identische Ergebnisse liefern.

In [10]:
seed = 11  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

## Docs

Wir generieren als nächsten die Dokumente, die das Eingangsmaterial für das Topic-Model bilden.

In [11]:
# Anzeige der Spaltennamen von data_model_1
print(data_model_1.columns.tolist())

data_model_1 = data_model_1[["veranstaltung_titel", "kursbeschreibung"]].copy()
data_model_1.head()

data_model_1 = data_model_1.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
data_model_1['titel_kursbesch'] = data_model_1['veranstaltung_titel'] + ' ' + data_model_1['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = data_model_1['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

['Unnamed: 0', 'veranstaltung_id', 'veranstaltung_titel', 'kursbeschreibung', 'kursformat_original', 'sprache_original', 'lernziele', 'lernmethode', 'literatur', 'voraussetzungen', 'zusatzinformationen', 'anmerkungen', 'pruefung', 'dozierende', 'teilnehmerzahl', 'sws', 'ects', 'url', 'nummer', 'pfad', 'scrape_datum', 'hochschule_id', 'hochschule_name', 'fakultaet_id', 'fakultaet_name', 'organisation_id', 'organisation_name', 'semester_id', 'semester_name', 'sprache_id', 'sprache_name', 'kursformat_id', 'kursformat_name', 'matchingart_id', 'matchingart_name', 'lehr_und_forschungsbereich_id', 'bio_info', 'ingen_info', 'medien_info', 'medizin_info', 'wirt_info', 'recht_info', 'geo_info', 'bau_info', 'base_info', 'model_1_b', 'model_2', 'model_3', 'model_4']


## Stopwords

In [25]:
from utils import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms)
irrelevant_terms

['vl',
 'übung',
 'übungen',
 'seminar',
 'arbeitsgruppenseminar',
 'oberseminar',
 'proseminar',
 'blockveranstaltung',
 'vorlesung',
 'kolloquium',
 'theoriekolloquium',
 'einführung',
 'tutorium',
 'ue',
 'vereinbarung',
 'projekt',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',
 'begleitseminar',
 'abschlussarbeiten',
 'unterrichtspraktikum',
 'masterseminar',
 'proseminare',
 'praxisseminar',
 'praxissemester',
 'schulpraxis',
 'ringpraktikum',
 'basispraktikum',
 'praxistage',
 'industriepraktikum',
 'vorkurs',
 'projektseminar',
 'juniorprofessur',
 'masterarbeiten',
 'forschungsseminar',
 'modulbeschreibung',
 'veranstaltung',
 'kommentare',
 'raum',
 'uhrzeit',
 'vereinbarung',
 'vorlesung',
 'übung',
 'beispiel',
 'studierende',
 'grundlage',
 'laborübung',
 'entwicklung',
 'funktion',
 'praktikum',
 'versuch',
 'methode',
 'modell',
 'kompetente',
 'teil'

# Model

Wir definieren unser Model.

In [26]:
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Current device:", model.device)

CUDA available: True
Current device: cuda:0


## Model-Settings, GPU-Check
Konfiguration frei wählbar (einfach im Code unten anpassen).
Hier kann eine erste explorative Untersuchung durchgeführt werden.

Wir checken weiterhin, ob wirklich unsere GPU verwendet wird. Wenn `cuda:0` ist dies der Fall.

In [ ]:
# CountVectorizer
vectorizer = CountVectorizer(
  stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
  token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
  ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
)

# Embedding Settings  
embedding_model = SentenceTransformer(
    "paraphrase-multilingual-mpnet-base-v2",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# UMAP Settings
umap_model = UMAP(
    n_neighbors=10,
    n_components=10,
    min_dist=0.0,
    metric="cosine",
    random_state=seed
)
# HDBSCAN Settings
hdbscan_model = HDBSCAN(
    min_cluster_size=15,
    cluster_selection_epsilon=0.2,
    prediction_data=True
)
# Representation Settings
representation_model = MaximalMarginalRelevance(diversity=0.1)

# BERTopic initialisieren
topic_model = BERTopic(
  embedding_model=embedding_model,
  #min_topic_size=10,
  nr_topics= 20, 
  language="multilingual",
  verbose=True,
  umap_model=umap_model,
  vectorizer_model=vectorizer,
  hdbscan_model=hdbscan_model,
  top_n_words = 15,
  representation_model=representation_model
)

## Training

Wir trainieren das Modell.

In [28]:
topic_model_1 = topic_model.fit(docs)
topic_model_1.get_topic_info()

2026-02-05 11:43:15,851 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 1602/1602 [00:32<00:00, 49.89it/s] 
2026-02-05 11:43:48,419 - BERTopic - Embedding - Completed ✓
2026-02-05 11:43:48,420 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-02-05 11:44:51,642 - BERTopic - Dimensionality - Completed ✓
2026-02-05 11:44:51,645 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-02-05 11:44:59,625 - BERTopic - Cluster - Completed ✓
2026-02-05 11:44:59,626 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2026-02-05 11:45:03,165 - BERTopic - Representation - Completed ✓
2026-02-05 11:45:03,174 - BERTopic - Topic reduction - Reducing number of topics
2026-02-05 11:45:04,611 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-02-05 11:45:25,696 - BERTopic - Representation - Completed ✓
2026-02-05 11:45:25,706 - BERTopic - Topic reduction -

,Topic,Count,Name,Representation,Representative_Docs
0,-1,8970,-1_gt_learning_gt lt_li gt,"[gt, learning, gt lt, li gt, informatik, engin...",[Grundlagen der Informationssicherheit/Datensi...
1,0,1464,0_softwaretechnik_software engineering_softwar...,"[softwaretechnik, software engineering, softwa...",[Übung: Softwaretechnik Übung 1 Die Vorlesung ...
2,1,844,1_bioinformatik_bioinformatik bioinformatik_bi...,"[bioinformatik, bioinformatik bioinformatik, b...","[Oberseminar Bioinformatik , Bioinformatik , ..."
3,2,730,2_informatik 2 01_tutorien informationssysteme...,"[informatik 2 01, tutorien informationssysteme...",[Oberseminar (IN2122) Abschluss-/Zwischenvortr...
4,3,702,3_datenbanken informationssysteme_datenbanksys...,"[datenbanken informationssysteme, datenbanksys...",[Visual Computing Praktikum: Ubiquitous Textte...
...,...,...,...,...,...
849,848,16,848_data science moodle_science moodle_01a dat...,"[data science moodle, science moodle, 01a data...","[inf-DSProj-01a: Data Science Projekt Moodle, ..."
850,849,16,849_causal_causality_causal inference_attack t...,"[causal, causality, causal inference, attack t...",[Causal Inference in Time Series (CIT4230006) ...
851,850,16,850_data knowlege_data knowlege engineering_kn...,"[data knowlege, data knowlege engineering, kno...","[Oberseminar zu Data & Knowlege Engineering , ..."
852,851,16,851_aud b algorithmen_b algorithmen datenstruk...,"[aud b algorithmen, b algorithmen datenstruktu...",[AI-AuD-B: Algorithmen und Datenstrukturen (Tu...


## Labeling

Mit Hilfe von gpt4o lassen wir uns die Topcis labeln. Das erleichtert die Interpretation des Modells

In [21]:
from openai import OpenAI as OpenAIClient
from bertopic.representation import OpenAI as OpenAIRep

client = OpenAIClient()  # nutzt OPENAI_API_KEY aus der Umgebung
rep = OpenAIRep(
    client=client, 
    model="gpt-4o-mini", 
    delay_in_seconds=10,      # Erhöhe Delay
    exponential_backoff=True, # Aktiviere das schrittweise längere Warten
    nr_docs=3                 # Weniger Dokumente pro Topic reduzieren die Token-Last
)
topic_model.update_topics(docs, representation_model=rep)
topic_model.get_topic_info() 

100%|██████████| 25/25 [04:32<00:00, 10.89s/it]


,Topic,Count,Name,Representation,Representative_Docs
0,-1,8965,-1_Data Science and Analytics,[Data Science and Analytics],[Einführung in die Informatik (ehemals Anwendu...
1,0,23778,0_Information Systems Education,[Information Systems Education],"[Software Engineering , Übung: Mathematische G..."
2,1,3510,1_Scientific Work Ethics,[Scientific Work Ethics],[KTR-Sem-M: Erwartete Teilnehmerzahl: 10 www: ...
3,2,3032,2_Robotics and AI,[Robotics and AI],[Dodo Alive! - Resurrecting the Dodo with Robo...
4,3,2384,3_Introduction to Bioinformatics,[Introduction to Bioinformatics],"[Querschnittsbereich Epidemiologie, Medizinisc..."
5,4,2348,4_Medical Image Processing,[Medical Image Processing],[Uebung - 3D Scanning & Motion Capture (IN2354...
6,5,1965,5_Internet of Things,[Internet of Things],[infIoT-01a: Internet of Things and Wireless N...
7,6,1760,6_Introduction to Cryptography,[Introduction to Cryptography],[Introduction to Modern Cryptography Cryptogra...
8,7,557,7_Signal Processing Basics,[Signal Processing Basics],[ Multirate Signal Processing The module offe...
9,8,506,8_Design Thinking in Business,[Design Thinking in Business],[Master-Seminar - Design Thinking for Business...


In [23]:
topic_model.get_topic_info() 

,Topic,Count,Name,Representation,Representative_Docs
0,-1,8965,-1_Data Science and Analytics,[Data Science and Analytics],[Einführung in die Informatik (ehemals Anwendu...
1,0,23778,0_Information Systems Education,[Information Systems Education],"[Software Engineering , Übung: Mathematische G..."
2,1,3510,1_Scientific Work Ethics,[Scientific Work Ethics],[KTR-Sem-M: Erwartete Teilnehmerzahl: 10 www: ...
3,2,3032,2_Robotics and AI,[Robotics and AI],[Dodo Alive! - Resurrecting the Dodo with Robo...
4,3,2384,3_Introduction to Bioinformatics,[Introduction to Bioinformatics],"[Querschnittsbereich Epidemiologie, Medizinisc..."
5,4,2348,4_Medical Image Processing,[Medical Image Processing],[Uebung - 3D Scanning & Motion Capture (IN2354...
6,5,1965,5_Internet of Things,[Internet of Things],[infIoT-01a: Internet of Things and Wireless N...
7,6,1760,6_Introduction to Cryptography,[Introduction to Cryptography],[Introduction to Modern Cryptography Cryptogra...
8,7,557,7_Signal Processing Basics,[Signal Processing Basics],[ Multirate Signal Processing The module offe...
9,8,506,8_Design Thinking in Business,[Design Thinking in Business],[Master-Seminar - Design Thinking for Business...


In [22]:
# Speichern
topic_model.save("models/explor_model_1", serialization="safetensors", save_embedding_model=True)

# Laden
from bertopic import BERTopic
loaded_model = BERTopic.load("models/explor_model_1")

2026-02-05 11:35:52,715 - BERTopic - WARNING: You are loading a BERTopic model without explicitly defining an embedding model. If you want to also load in an embedding model, make sure to use `BERTopic.load(my_model, embedding_model=my_embedding_model)`.
